# Install & Import Libraries

# BERTopic per Year

In [5]:
!pip install bertopic[all] umap-learn

In [6]:
from bertopic import BERTopic
from umap import UMAP
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
import pandas as pd

In [ ]:
df_debates = pd.read_csv('debate_transcripts_cleaned.csv', encoding='utf-8')
df_debates = df_debates[df_debates["is_candidate"] == True].copy()

In [ ]:
def run_bertopic_over_metadata(df, groupby_col="year", model=None, embedding_model="all-MiniLM-L6-v2", verbose=True):
    """
    Fits BERTopic to the 'dialogue' column of df and returns topic model and topics over time/group.

    Parameters:
        df (pd.DataFrame): DataFrame with at least 'dialogue' and one metadata column
        groupby_col (str): Column in df to group topic evolution by. Options: "year", "date", "debate_title", "actor", "party"
        model (BERTopic or None): Pass an existing BERTopic model to reuse, or None to create new
        embedding_model (str or SentenceTransformer): Sentence embedding model name or object
        verbose (bool): Whether to print status messages

    Returns:
        topic_model (BERTopic)
        topics_over_group (pd.DataFrame)
        topics (List[int])
        probs (List[float])
    """

    assert "dialogue" in df.columns, "DataFrame must have a 'dialogue' column."
    assert groupby_col in df.columns, f"{groupby_col} not found in DataFrame columns."

    docs = df["dialogue"].tolist()
    timestamps = df[groupby_col].astype(str).tolist()  # convert to string for topics_over_time

    if model is None:
        if verbose:
            print(f"Creating new BERTopic model using embedding model: {embedding_model}")
        model = BERTopic(
            language="english",
            embedding_model=embedding_model,
            verbose=verbose
        )

    if verbose:
        print("Fitting BERTopic...")
    topics, probs = model.fit_transform(docs)

    if verbose:
        print(f"Computing topic evolution over '{groupby_col}'...")
    topics_over_group = model.topics_over_time(docs, timestamps)

    return model, topics_over_group, topics, probs


2025-05-21 20:33:23,981 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/262 [00:00<?, ?it/s]

In [ ]:
model, topics_yearly, topics, probs = run_bertopic_over_metadata(df_debates, groupby_col="year")
model.visualize_topics_over_time(topics_yearly)